In [421]:
#compute the eigenvectors of Schur's matrix according to Morton's '78 paper
#"On the Eigenvectors of Schur's Matrix": https://www.sciencedirect.com/science/article/pii/0022314X80900839
#NOTE: The DFT matrix is subtly different than the Schur matrix. DFT = exp(2*pi*i*m*n/q) for 0 <= m,n <= q-1
#whereas for Schur's matrix it is 1 <= m,n <= q

In [433]:
#compute the eigenvectors of the unitary matrix corresponding to the Fourier transform over the complex numbers
q=4
alpha = exp(2*pi*I/q)
#K.<alpha> = CyclotomicField(3)
U = matrix([[alpha**(i*j) for j in range(1,q+1)] for i in range(1,q+1)])
U.eigenvectors_right()

[(2*I, [(1, 0, -1, 0)], 1),
 (-2, [(1, 1, 1, -1)], 1),
 (2, [(1, 0, 1, 2), (0, 1, 0, 1)], 2)]

In [378]:
#define the vectors X_d given a divisor d of q/f(chi), where f(chi) is the conductor of the character \chi
def X(d,chi):
    q = chi.modulus()
    assert d.divides(q/chi.conductor())
    return vector([chi(n/d) if d.divides(n) else 0 for n in range(1,q+1)])

In [ ]:
#compute eigenvectors given a character chi and a divisor d of q/f(chi)
def eigenvector(d,chi):
    try:
        assert d.divides(q/chi.conductor())
    except AssertionError:
        print("Error: d must divide q/f(chi)")
        return

    #define the eigenvectors E(chi,d,lambda) as in (13) in the paper
    conjugate_divisor = Integer(chi.modulus()/(chi.conductor()*d))
    eig_plus = sqrt(chi(-1)*chi.modulus())
    eig_minus = -sqrt(chi(-1)*chi.modulus())
    #need to restrict character to modulus conductor, i.e. the primitive character
    E = lambda chi, d, eig: sqrt(d)*X(d,chi) + (eig/(chi.bar().restrict(chi.conductor()).gauss_sum()*sqrt(d)))*conjugate(X(conjugate_divisor,chi))

    #check if chi is real and if the ratio of modulus to conductor is the square of a divisor
    reality_condition = chi.bar() == chi
    square_condition = d^2 == chi.modulus()/chi.conductor()

    #if chi is real and square condition is met, just return X_d(chi)
    if reality_condition and square_condition:
        return X(d,chi)

    #if one or the other condition is met, return a pair of eigenvalues corresponding to E(chi,d,\pm lambda)
    if reality_condition != square_condition:
        return E(chi,d,eig_plus), E(chi,d,eig_minus)

    #if neither condition are met, return four eigenvalues E(chi,d,\pm lambda) and E(chi_bar,d,\pm lambda)
    if (not reality_condition) and (not square_condition):
        return E(chi,d,eig_plus), E(chi,d,eig_minus), E(chi.bar(),d,eig_plus), E(chi.bar(),d,eig_minus)

In [463]:
G = DirichletGroup(q); G

Group of Dirichlet characters modulo 4 with values in Cyclotomic Field of order 2 and degree 1

In [464]:
len(G)

2

In [436]:
e = G[0]; f = G[1]

In [475]:
#note we are obtaining Gauss sums of zero when q=4 and d=1
#the Gauss sum in the paper is really a restricted sum
#it's only summing using a primitive f^th root of unity where f = f(chi) is the conductor
print(f.bar().gauss_sum())
print(f.restrict(f.conductor()).gauss_sum())

2*zeta4
2*zeta4


In [466]:
f

Dirichlet character modulo 4 of conductor 4 mapping 3 |--> -1

In [459]:
f.primitive_character().gauss_sum()

2*zeta4

In [468]:
eigenvector(2,e)

(0, 1, 0, 0)

In [406]:
eig_plus = sqrt(e(-1)*e.modulus()); print(eig_plus)
eig_minus = -sqrt(e(-1)*e.modulus()); print(eig_minus)

sqrt(3)
-sqrt(3)


In [412]:
conjugate_divisor = Integer(e.modulus()/(e.conductor()*1)); conjugate_divisor

3

In [408]:
(eig_plus/(e.bar().gauss_sum()*sqrt(1)))

-sqrt(3)

In [409]:
e.bar().gauss_sum()

-1

In [430]:
conjugate(X(3,e))

(0, 0, 1)

In [432]:
sqrt(1)*X(1,e) + (-sqrt(3))*X(3,e)

(1, 1, -sqrt(3))